# COGS 108 - EDA Checkpoint

# Names
- Nathan Ahmann
- Alex Guan
- Alan Miyazaki
- Renaldy Herlim

<a id='research_question'></a>
# Research Question

Does crime happen more or less frequently around police stations and can that knowledge be used to more effectively distribute police station locations?

# Setup

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import datetime

import geopandas as gpd

# Data Cleaning

Since the two crime datasets were from the same organization (LAPD), the two sets were really easy to combine. The only change that needed to be done was that the dataset from 2010-2019 had a space in column name for Area while the dataset for 2020-present did not. Aside from this, all other columns were the same and the datasets were easily combined.

In [ ]:
# Reading datasets from two different time periods

# https://data.lacity.org/Public-Safety/Crime-Data-from-2010-to-2019/63jg-8b9z
past_df = pd.read_csv("Crime_Data_from_2010_to_2019.csv", dtype = {"TIME OCC": str})
# https://data.lacity.org/Public-Safety/Crime-Data-from-2020-to-Present/2nrs-mtv8
present_df = pd.read_csv("Crime_Data_from_2020_to_Present.csv", dtype = {"TIME OCC": str})

LAPD_df = pd.read_csv("LAPD_Police_Stations.csv")

# past dataset has column name typo
past_df = past_df.rename(columns={"AREA ": "AREA"})

# Both datasets use the same columns 
df = pd.concat([past_df, present_df])

The crime dataset had quite a few columns that were unnecessary to us. Mainly these were columns containing codes that law enforcement used internally. However, these columns usually had a description column in the dataset so the code column was unnecessary for us. The only exception were Mocodes and Part 1-2 which are internally used codes but did not contain an associated description column. However after looking into what these codes meant, it was decided that they did not provide any use to us and were removed. In addition to this, we also removed Date Rptd, date crime was reported, since we only care about when the crime occured and not when it was reported. We also replaced the codes in Vict Descent with their actual descriptions that was provided in the site the dataset came from.

In [ ]:
# Dropping code columns since these are internal use and we don't get much 
# value from them given we have their description in a seperate column
df = df.drop(columns=[
    "Crm Cd", "Crm Cd 1", "Crm Cd 2", "Crm Cd 3", 
    "Crm Cd 4", "Premis Cd", "Weapon Used Cd", 
    "Mocodes", "Part 1-2", "Status", "Date Rptd"
    ])

# Changed code to be readable description
df["Vict Descent"] = df["Vict Descent"].replace({
                            "A": "Other Asian", 
                            "B": "Black", 
                            "C": "Chinese", 
                            "D": "Cambodian", 
                            "F": "Filipino", 
                            "G": "Guamanian", 
                            "H": "Hispanic/Latin/Mexican", 
                            "I": "American Indian/Alaskan Native", 
                            "J": "Japanese", 
                            "K": "Korean", 
                            "L": "Laotian", 
                            "O": "Other", 
                            "P": "Pacific Islander", 
                            "S": "Samoan", 
                            "U": "Hawaiian", 
                            "V": "Vietnamese", 
                            "W": "White", 
                            "X": "Unknown", 
                            "Z": "Asian Indian",
                        })

Additionally, we noticed that some of the longitudes and latitudes weren't in Los Angeles when we tried to plot them. Specifically a lot of data was at 0,0. While we could possibly figure out an estimated longitude and latitude since they still had data in the location column, we instead decided to drop them since there was not a large amount.

In [ ]:
before_size = df.shape[0]
df = df.drop(df.loc[df['LAT'] < 10].index)
df = df.drop(df.loc[df['LON'] > -10].index)
after_size = df.shape[0]
print('Before: ', before_size)
print('After: ', after_size)
print('Dropped',before_size - after_size,'rows, which is',(before_size - after_size) / before_size,'% of the entire dataset')

Lastly, we needed to clean up the date and time columns

add datetime conversion
maybe rename some columns for neatness/readability

To combine our geospatial datasets and make them work with our crime dataset, we had to rename some columns and do some merges. First, we changed the X, Y system used on the LAPD dataset to longitude, latitude to match the crime dataset naming convention.

Additionally, we had to make use of geopandas to read in the precinct area data. This dataset just contains shapes and areas for each precinct and is used for plotting and area data.

In [ ]:
# Renamed X,Y to Longitude, Latitude
LAPD_df = LAPD_df.rename(columns={
    "X": "Longitude", 
    "Y": "Latitude"
    })

dont forget to redo dataset section on final ver to add in new map dataset. Didn't have previously but need for plotting  

Later on in our EDA and analysis we do some additional operations that could be considered data cleaning. These were done for calculated fields or for specific plotting/analysis purposes. We left them in their own sections to make the code easier to follow and because our dataset at this point represents a cleaned dataset that others could use for their own purposes.

# Data Analysis & Results (EDA)

The first thing to always do is take a look at a few rows of the dataset. This gives us an idea of what our dataset looks like after cleaning and what we could explore more. The police station and divisions datasets are used for geographic data and plotting so those will be looked at in those sections.

In [ ]:
df.head()

One thing that we would be interested in knowing is if there is an equal distribution of crimes between police stations. From this we can see that there is a difference between how many cases each station has handled. This could either mean that some police stations are responsible for a larger area or there are more crimes happening in the areas those stations are covering. 

In [ ]:
import matplotlib.pyplot as plt

Just from a quick glance of the first few rows we can see the type of data we have access to.

We can also see that there are some NaN values, at least in Weapon Desc (which makes sense if the crime did not involve a weapon) and in Cross Street.

Lastly, we have some columns that won't be useful for us.

Since most of our columns are non-numeric, looking at df.describe() won't be super helpful and instead we need to find our own metrics to look at.

In [ ]:
print('Number of unique crime descriptions: ', df['Crm Cd Desc'].nunique())
print('Number of unique victim descents: ', df['Vict Descent'].nunique())
print('Number of unique victim sexes: ', df['Vict Sex'].nunique())
print('Number of unique status descriptions: ', df['Status Desc'].nunique())
print('Number of unique premise descriptions: ', df['Premis Desc'].nunique())
print('Number of unique area names (should be 21 which corresponds with Precincts): ', df['AREA NAME'].nunique())

Check to see if Longitude and Latitude make sense for LA area. (Originally included 0,0 values, now seems to include the expected area)

In [ ]:
pd.DataFrame(columns=['Column','Min', 'Mean', 'Median', 'Max'], data=[
    ['LAT', df['LAT'].min(), df['LAT'].mean(), df['LAT'].median(), df['LAT'].max()],
    ['LON', df['LON'].min(), df['LON'].mean(), df['LON'].median(), df['LON'].max()]
])

Even after removing the 0,0 longitude latitude crimes, many crimes happen outside of the zone of any of the precincts (see the plot below). Since we plan to work with location data and clustering, these outliers will cause problems for our analysis. So we removed even more of the data to hone in on the area we truly care about, which is within the precinct boundaries.

In [ ]:
map_df = gpd.read_file('LAPD_Divisions.shp')

fig, ax = plt.subplots(1, figsize = (20, 12))
map_df.boundary.plot(color='black', ax=ax)
df.plot(ax=ax, x='LON', y='LAT', kind='scatter', c='AREA',  s=0.1, cmap='viridis')
LAPD_df.plot(ax=ax, x='Longitude', y='Latitude', kind='scatter', s=5, c='black')

In [ ]:
# code snippet that could be useful later for determining better bounds
# map_df['geometry'].astype(str)\
#     .str[8:]\
#     .str.replace('(', '')\
#     .str.replace(')', '')\
#     .str.replace(',', '')\
#     .str.split(' ')

In [ ]:
before_size = df.shape[0]

# arbitrarily chosen bounds for LA area
df = df.drop(df.loc[(df['LAT'] <= 33.7) & (df['LAT'] >= 34.37) & (df['LON'] <= -118.7) & (df['LON'] >= -118.15)].index)

after_size = df.shape[0]
print('Before: ', before_size)
print('After: ', after_size)
print('Dropped',before_size - after_size,'rows, which is',(before_size - after_size) / before_size,'% of the entire dataset')

So now our dataset just contains the areas within the precinct and not a large area around them. This will help with our clustering and our analysis by removing outlier datapoints. These points outside the precinct boundaries seem randomly assigned to precincts or at least use a pattern that was not noted.

With this all finished we can plot the whole area again. Additionally we can add each of the latitudes and longitudes from the entire crime set.

In [ ]:
map_df = gpd.read_file('LAPD_Divisions.shp')
df_plot = df.loc[(df['LAT'] >= 33.7) & (df['LAT'] <= 34.37) & (df['LON'] >= -118.7) & (df['LON'] <= -118.15)]

fig, ax = plt.subplots(1, figsize = (20, 12))
map_df.boundary.plot(color='black', ax=ax)
df_plot.plot(ax=ax, x='LON', y='LAT', kind='scatter', c='AREA',  s=0.1, alpha=0.5, cmap='viridis')
LAPD_df.plot(ax=ax, x='Longitude', y='Latitude', kind='scatter', s=5, c='black')

So the crime distribution is largely containted within the precincts. The black dots are the police station locations themselves. A good thing to take note of is that the police stations are not centered in their respective zones. While some are roughly in the middle, there are also some on the edge of their zones. This is interesting since for effective coverage we would expect them to be centered so they can optimally cover their area. This will likely affect our analysis since our calculated locations will be centered on the crime around them and thus would likely be centered in their zones.

Crime Counts in Relation to Precinct

Since our research question involves location information we want to focus our explorations around that. Specifically we would like to see if there are similar amounts of crimes commited in each precinct, if each precinct has similar areas, if each precinct has similar coverage.

First let's look at the number of crimes in each precinct.

One thing that we would be interested in knowing is if there is an equal distribution of crimes between police stations. From this we can see that there is a difference between how many cases each station has handled. This could either mean that some police stations are responsible for a larger area or there are more crimes happening in the areas those stations are covering.

In [ ]:
# setting up a merged crime count table that has counts and area of precincts using df and map_df
crime_counts = map_df.merge(df.groupby('AREA')[['DR_NO']].count().rename(columns={'DR_NO':'CRIME_COUNT'}), left_on='PREC', right_on='AREA')
crime_counts = crime_counts.sort_values(by='CRIME_COUNT', ascending=False)
crime_counts.plot(kind='barh', x='APREC', y='CRIME_COUNT')

While there are a few with less and a few with more it seems that there are similar amounts of crime commited in each precint.

But based on our map we know that the precint's are not all the same size, so can the differences in numbers of crime commited be related to size?

In [ ]:
crime_counts['CRIME_COUNT_VS_AREA'] = crime_counts['CRIME_COUNT'] / crime_counts['AREA']

crime_counts['scaled_AREA'] = crime_counts['AREA'] / crime_counts['AREA'].max()
crime_counts['scaled_CRIME_COUNT'] = crime_counts['CRIME_COUNT'] / crime_counts['CRIME_COUNT'].max()
crime_counts['scaled_CRIME_COUNT_VS_AREA'] = crime_counts['CRIME_COUNT_VS_AREA'] / crime_counts['CRIME_COUNT_VS_AREA'].max()
crime_counts.plot(
    kind='barh', 
    x='APREC', 
    y=['scaled_CRIME_COUNT','scaled_AREA','scaled_CRIME_COUNT_VS_AREA'], 
    subplots=True, layout=(1,3), 
    figsize=(10, 5), 
    sharex=True, sharey=True,
    legend=False,
    title=['Number of Crimes Committed', 'Area of Precinct', 'Number of Crimes Committed/ Area']
)

All of the values have been scaled
The first bar chart is just the number of crimes commited in each area.
The second is the area of each precint.
The last bar chart is the number of crimes / the area of the precinct.
Since the axis are aligned each corresponds to the same area. So despite having an average number of crimes commited, the Central precinct has a very small area compared to the others and gets a higher value on the right bar chart. Since the bars on the right are not all the same size, then that means there is a disparity between the size of a precint and the number of crimes commited there.

Time EDA below, not sure if useful for final version


We also wanted to look into the time that crimes happen. The easiest firs thing to look at is the number of crimes committed at each time.

In [ ]:
df["TIME OCC"].str[:2].value_counts().sort_index().plot(xticks=[i for i in range(24)])

This is to see when do most crimes happen. From this we can see that the most common hour for a crime to occur over the past 13 years was at noon. The least common time for a crime to occur was at 5 am. However this is a massive spike at 12 which is odd. A reason for this could be that the dataset only counts crimes reported which means that since it is noon, more people are out and can see/report on a crime. In contrast, people are sleeping at night so there are less people reporting crimes. Another reason for this could be that when a crime happens, they are more focused on the criminal instead of what time it is so when they make a report to an officer, they may choose noon due to it being daylight.

After looking at time, we also wanted to look at it on a date basis.

In [ ]:
pd.to_datetime(df["DATE OCC"]).value_counts().sort_index().plot()

In [ ]:
pd.to_datetime(df["DATE OCC"]).value_counts()[:5]

In [ ]:
pd.to_datetime(df["DATE OCC"]).dt.to_period('M').value_counts().sort_index().plot()

Nothing major stood out in this plot, except that there were some random drops on about a yearly basis and some overall trends of more crime during 2017-2019. Addtionally there seems to be a drop in crime from 2019-2021. This could be due to COVID and stay in place mandates. Lastly, 2023 has a giant drop but this is just because it is currently taking in more data.

And graphed on just an aggregated yearly basis (ignored 2023 this time for a better zoom):

In [ ]:
pd.to_datetime(df["DATE OCC"]).dt.to_period('Y').value_counts().sort_index()[:-1].plot()

Doesn't really show anything different than above. Just similar drop around 2019-2021.

In [ ]:
df["AREA NAME"].value_counts().plot(kind="barh")

We also want to know what types of crimes are being commmitted, because of the varying types of crimes we want to group on some key words to further understand it in simple terms

In [ ]:
df["Crm Cd Desc"].unique()

In [ ]:
len(df["Crm Cd Desc"].unique())

144

In [ ]:
#Analyze some common types of crimes

df['CRIME_TYPE_GROUP'] = df['Crm Cd Desc'].apply(lambda x: 'Petty' if 'PETTY' in x 
                                        else ('Child' if 'CHILD' in x 
                                        else ('Sexual' if 'SEX' in x 
                                        else ('Battery' if 'BATTERY' in x 
                                        else ('Assault' if 'ASSAULT' in x 
                                        else 'Other')))))

df['CRIME_TYPE_GROUP'].value_counts()

Analyze the demographic of victims to get a better understanding of our data

In [ ]:
#Analyze Victim AGE
df_age = df[df['Vict Age'] > 0 ]['Vict Age']

plt.bar(df_age.value_counts().index, df_age.value_counts().values)

# Add labels and a title
plt.xlabel('Age')
plt.ylabel('Count')
plt.title('Counts of Crime Victims by Age')

There are more Male victims than Female victims

In [ ]:
df_vict_d = df['Vict Descent']

In [ ]:
plt.bar(df_vict_d.value_counts().index, df_vict_d.value_counts().values)

# Add labels and a title
plt.xlabel('Victim Race')
plt.xticks(rotation = 90)
plt.ylabel('Count')
plt.title('Counts of Crime Victims by Race')

In [ ]:
#Analyze Victim SEX
df_vict_s = df[(df['Vict Sex'] == 'M')| (df['Vict Sex'] == 'F')]['Vict Sex']

plt.bar(df_vict_s.value_counts().index, df_vict_s.value_counts().values)

# Add labels and a title
plt.xlabel('Victim Sex')
plt.ylabel('Count')
plt.title('Counts of Crime Victims by Sex')

## Analysis of Crime Time Occurances

In [ ]:
df["TIME OCC"].value_counts()

This is to see when do most crimes happen. From this we can see that the most common hour for a crime to occur over the past 13 years was at noon. The least common time for a crime to occur was at 5 am. However this is a massive spike at 12 which is odd. A reason for this could be that the dataset only counts crimes reported which means that since it is noon, more people are out and can see/report on a crime. In contrast, people are sleeping at night so there are less people reporting crimes. Another reason for this could be that when a crime happens, they are more focused on the criminal instead of what time it is so when they make a report to an officer, they may choose noon due to it being daylight.

In [ ]:
df["TIME OCC"].str[:2].value_counts().sort_index().plot()

In [ ]:
df["TIME OCC"].str[:2].value_counts()

Group times into "morning" , "afternoon", "evening", and "night"

In [ ]:
df["TIME OCC"].str[:2].astype(int)

In [ ]:
# Define a dictionary to map the times to time of day categories
time_dict = {'Morning': range(5, 12),
             'Afternoon': range(12, 18),
             'Evening': range(18, 24),
             'Night': range(0,5)}

# Use the pandas map function to create a new column with the time of day categories
df['TIME_OCC_GROUP'] = df["TIME OCC"].str[:2].astype(int).map(lambda x: next((k for k, v in time_dict.items() if int(x) in v), None))

In [ ]:
# Create a barplot using matplotlib
plt.bar(df['TIME_OCC_GROUP'].value_counts().index, df['TIME_OCC_GROUP'].value_counts().values)

# Add labels and a title
plt.xlabel('Time of Day')
plt.ylabel('Count')
plt.title('Counts of Crime by Time of Day')

# Show the plot
plt.show()

In [ ]:
df["TIME OCC"].value_counts()

In [ ]:
df["DATE OCC"].value_counts().sort_index()

In [ ]:
pd.to_datetime(df["DATE OCC"]).value_counts().sort_index().plot()

In [ ]:
df_sex = df[(df['Vict Sex'] == 'M')| (df['Vict Sex'] == 'F')]
df_sex = pd.DataFrame(df_sex.groupby(['Vict Sex', 'TIME_OCC_GROUP']).count()['TIME OCC'])
df_sex

Seems like there are more crimes happening during new years?

In [ ]:
df.head()

In [ ]:
pd.to_datetime(df["DATE OCC"]).value_counts()

Number of crimes reported plotted by month and year

In [ ]:
pd.to_datetime(df["DATE OCC"]).dt.to_period('M').value_counts().sort_index().plot()

In [ ]:
pd.to_datetime(df["DATE OCC"]).dt.to_period('M').value_counts()

In which month of the year is crime typically higher?

In [ ]:
months = pd.to_datetime(df["DATE OCC"]).dt.month

# Create a barplot using matplotlib
plt.bar(months.value_counts().index, months.value_counts().values)

# Add labels and a title
plt.xlabel('Month')
plt.ylabel('Count')
plt.title('Counts of Crime by Month')

# Show the plot
plt.show()

It looks like crime is pretty much equally spread out throughout the year, except for January.

This is number of crims reported over years. 2023 is extremely low because the year is going on.

In [ ]:
pd.to_datetime(df["DATE OCC"]).dt.to_period('Y').value_counts().sort_index().plot()

In [ ]:
pd.to_datetime(df["DATE OCC"]).dt.to_period('Y').value_counts()

Removing 2023 to get a better graph. We can see some oddities. 2013, 2014, and 2020 are low in comparison to other years. 2020 is likely due to the pandemic since everyone was encouraged to stay at home which leads to less crimes reported (and maybe less crimes happening?). I have no idea of any reason to explain why 2013 and 2014 are so low. In addition its a bit weird that crime starts picking to make a curve at 2017. Also a massive spite at 2022 as well but not too sure of the reason there. 

In [ ]:
pd.to_datetime(df["DATE OCC"]).dt.to_period('Y').value_counts().sort_index()[:-1].plot()

In [ ]:
LAPD_df.head()

## Looking at New Years data from 2010 to 2023

As discussed earlier, we noticed that there appears to be spikes in criminal activity on each of the past 13 New Years days. Due to how massive the disparity of the crime rates appears to be, it may be interesting to look at these days and uncover what kinds of crimes may be causing such a massive jump. We will start by copying the original dataframe and adding an additional column which transforms the 'DATE OCC' (date and time) column into a datetime column to easily extract January 1st from each year.

In [ ]:
# Make a copy of the dataframe and create a new one with a datetime column for easy filtering
datetime_col = pd.to_datetime(df['DATE OCC'])
new_df = df.copy()
new_df['Date'] = datetime_col
new_df.head()

We can now easily filter the dataframe by making Month and Day equal 1, which will only contain data from New Years days.

In [ ]:
# Create new dataframe that only contains crimes on New Years Day from 2010 - 2023
new_years = new_df[(new_df['Date'].dt.month == 1) & (new_df['Date'].dt.day == 1)]
new_years.head()

In [ ]:
new_years.shape

We notice here that the data from New Years days contains over 20,000 crimes, which is about 0.72% of the entire dataset. This comes to an average of about 1559 crimes per New Years day, compared to the daily average of crimes being only 628. It appears that about 2.5 times more crime is reported on a single New Years day compared to any other day in LA.

In [ ]:
# Average New Years day
20262/13

In [ ]:
# Total days from January 2010 - March 2023
365 * 12 + (80)

In [ ]:
# Average number of crimes per day from Jan 2010 - March 2023
df.shape[0] / 4460

#### Looking at the top crimes on a given New Years day

In [ ]:
# Looking at the top 20 crimes committed on New Years days
top_20 = new_years['Crm Cd Desc'].value_counts()[:20]
top_20

In [ ]:
top_20.sort_values().plot(kind = 'barh')

#### Identity theft appears to be the overwhelming majority of crimes on New Years days over the past 13 years, with no other crime coming close to its prevelance. This makes some logical sense, due to fake ID's being used for the purchase of alcohol, bar hopping, and as defense mechanisms for avoiding a DUI."